In [ ]:
%cd /content/drive/MyDrive/01.Projects/cse257sp21/

/content/drive/MyDrive/01.Projects/cse257sp21


###Dividing input texts for Experiments

In [ ]:
filecount = 50
line_per_file = 10
infile = "DUC2004_original.txt"
duc2004 = []
with open(infile, 'r') as f:
  duc2004 = f.readlines()

for c in range(filecount):
  filename = f"./DUC2004/DUC2004_{c+1:02d}.txt"
  with open(filename, 'w') as f:
    f.writelines(duc2004[c*line_per_file:(c+1)*line_per_file])

### Combining result texts

In [ ]:
#filecount = 24
#outfile = f"NOTS_all.txt"
kwnum = 2
filecount = 7

#outfile = f"KTMT_{kwnum}KW_all.txt"
outfile = f"TSKW{kwnum}_new_all.txt"
wf = open(outfile, 'w')

for c in range(filecount):
    #filename = f"./NOTS/NOTS_{c+1:02d}.txt"
    #filename = f"./KTMT_{kwnum}KW/mtkw{kwnum}_{c+1}.txt"
    filename = f"./TSKW/tskw{kwnum}_new_{c+1:02d}.txt"
    #print(filename)
    with open(filename, newline='') as f:
      lines = f.read()
      wf.write(lines)

wf.close()

In [ ]:
const = 5
filecount = 46
outfile = f"NOMT{const:02d}_all.txt"
wf = open(outfile, 'w')

for c in range(filecount):
    filename = f"./NOMT_const_{const:02d}/NOMT{const:02d}_{c+1:02d}.txt"
    with open(filename, newline='') as f:
      lines = f.read()
      wf.write(lines)

wf.close()

"""
import csv

with open(f"NOMT{const}_result.csv", 'w', newline='') as csvfile:
  filednames = ['org src', 'new src', '-', 'new pred', '-', 'org pred']
  writer = csv.DictWriter(csvfile, filednames=filednames)
  writer.writeheader()

  for c in range(filecount):
    filename = f"NOMT{const}_{c+1:02d}.txt"
    with open(filename, newline='') as f:
      totallines.append(f.readlines)
  



import csv

with open('names.csv', 'w', newline='') as csvfile:
    fieldnames = ['first_name', 'last_name']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerow({'first_name': 'Baked', 'last_name': 'Beans'})
    writer.writerow({'first_name': 'Lovely', 'last_name': 'Spam'})
    writer.writerow({'first_name': 'Wonderful', 'last_name': 'Spam'})
"""

'\nimport csv\n\nwith open(f"NOMT{const}_result.csv", \'w\', newline=\'\') as csvfile:\n  filednames = [\'org src\', \'new src\', \'-\', \'new pred\', \'-\', \'org pred\']\n  writer = csv.DictWriter(csvfile, filednames=filednames)\n  writer.writeheader()\n\n  for c in range(filecount):\n    filename = f"NOMT{const}_{c+1:02d}.txt"\n    with open(filename, newline=\'\') as f:\n      totallines.append(f.readlines)\n  \n\n\n\nimport csv\n\nwith open(\'names.csv\', \'w\', newline=\'\') as csvfile:\n    fieldnames = [\'first_name\', \'last_name\']\n    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)\n\n    writer.writeheader()\n    writer.writerow({\'first_name\': \'Baked\', \'last_name\': \'Beans\'})\n    writer.writerow({\'first_name\': \'Lovely\', \'last_name\': \'Spam\'})\n    writer.writerow({\'first_name\': \'Wonderful\', \'last_name\': \'Spam\'})\n'

### Analyzing Result File (CSV)

In [ ]:
import csv
import numpy as np
from nltk.translate.bleu_score import sentence_bleu

org_src_list = [] #np.zeros((500,1))
new_src_list = [] #np.zeros((500,1))
org_pred_list = [] #np.zeros((500,1))
new_pred_list = [] #np.zeros((500,1))

const = 10

#target_file = f"210518_result_data_NOMT500_const{const:02d}.csv"
#target_file = "DUC2004_result_200.csv"
#target_file = "KTMT_1KW.csv"
target_file = "TSKW_3.csv"

with open(target_file, newline='') as csvfile:
  reader = csv.reader(csvfile, delimiter = ',', quotechar='"')
  for org_src, new_src, _, new_pred, _, org_pred in reader:
    org_src_list.append(org_src)
    new_src_list.append(new_src)
    org_pred_list.append(org_pred)
    new_pred_list.append(new_pred)

# trim the header
org_src_list = org_src_list[1:]
new_src_list = new_src_list[1:]
org_pred_list = org_pred_list[1:]
new_pred_list = new_pred_list[1:]

print(f"org_src: {len(org_src_list)}, new_src: {len(new_src_list)}, org_pred: {len(org_pred_list)}, new_pred: {len(new_pred_list)}")

org_src: 70, new_src: 70, org_pred: 70, new_pred: 70


#### Overlapping Attack Result Analysis

In [ ]:
# overlapping count excluding words
#overlap_count_exclude = set(["<unk>", "Ein", "ein", "und", ".", ",", "der", "die", "das", "oder"])   ## for Machine Translation
overlap_count_exclude = set(["<unk>", "a", "an", "the", ".", ",", "and", "or"])   ## for Text Summarization

src_len_list = np.zeros((len(org_src_list)), dtype=int)
pred_len_list = np.zeros((len(org_pred_list)), dtype=int)
src_diff_list = np.zeros((len(org_src_list)), dtype=int)
pred_same_list = np.zeros((len(org_pred_list)), dtype=int)

nonoverlapping_success_count = 0
src_diff_total = 0
src_diff_success = 0

src_diff_percentage_total = 0.0
src_diff_percentage_success = 0.0

pred_vocab_success = 0

bleu_score_total = 0.0

for i in range(len(org_src_list)):
  org_src = org_src_list[i]
  new_src = new_src_list[i]
  org_pred = org_pred_list[i]
  new_pred = new_pred_list[i]

  # compare src, pred diff
  org_src_words = org_src.split(" ")
  new_src_words = new_src.split(" ")

  # calculate BLEU score
  score = sentence_bleu([org_src_words], new_src_words)
  bleu_score_total += score

  src_len_list[i] = len(org_src_words)
  assert(len(org_src_words)==len(new_src_words))
  src_diff_count = 0
  for j in range(src_len_list[i]):
    if org_src_words[j] != new_src_words[j]:
      src_diff_count += 1
  
  src_diff_list[i] = src_diff_count
  src_diff_total += src_diff_count

  src_diff_percentage = src_diff_count / src_len_list[i] * 100.0
  src_diff_percentage_total += src_diff_percentage

  # compare pred diff
  org_pred_words = org_pred.split(" ")
  new_pred_words = new_pred.split(" ")
  pred_len_diff = abs(len(org_pred_words) - len(new_pred_words))
  pred_same_count = 0
  for j in range(min(len(org_pred_words), len(new_pred_words))):
    if org_pred_words[j] == new_pred_words[j]:
      if org_pred_words[j] not in overlap_count_exclude:
        pred_same_count += 1   
  
  pred_same_list[i] = pred_same_count

  #print(f"src diff: {src_diff_count}, percentage: {src_diff_count/org_src_wc*100:.2f}")
  print(f"src diff: {src_diff_count}, pred same: {pred_same_count}")
  if pred_same_count == 0:
    nonoverlapping_success_count += 1
    src_diff_success += src_diff_count
    src_diff_percentage_success += src_diff_percentage
  #if pred_same_count == 1:
    #print(f"org pred: {org_pred}\nnew_pred: {new_pred}")

  # compare pred vocab diff
  org_pred_set = set(org_pred_words).difference(overlap_count_exclude)
  new_pred_set = set(new_pred_words).difference(overlap_count_exclude)
  pred_new_vocab = new_pred_set.difference(org_pred_set)
  
  #print(len(pred_new_vocab), pred_new_vocab)
  if len(new_pred_set) == len(pred_new_vocab):
    pred_vocab_success += 1
    print(org_pred_set)
    print(new_pred_set)

print(f"nonoverlapping_success_count: {nonoverlapping_success_count}, success %: {nonoverlapping_success_count/len(org_src_list)*100:.2f}%")
print(f"avg src diff for all: {sum(src_diff_list)/len(src_diff_list):.2f}")
print(f"avg src diff percentage for all: {src_diff_percentage_total/len(src_diff_list):.2f}%")
print(f"avg src diff for success: {src_diff_success / nonoverlapping_success_count :.2f}")#len(src_diff_list)}")
print(f"avg src diff percentage for success: {src_diff_percentage_success/nonoverlapping_success_count:.2f}%")
print(f"vocab success: {pred_vocab_success}, rate: {pred_vocab_success/len(org_src_list)*100:.2f}%")
print(f"bleu score: {bleu_score_total/len(org_src_list):.3f}")

src diff: 1, pred same: 7
src diff: 1, pred same: 4
src diff: 1, pred same: 7
src diff: 1, pred same: 4
src diff: 0, pred same: 7
src diff: 0, pred same: 6
src diff: 1, pred same: 8
src diff: 0, pred same: 5
src diff: 1, pred same: 8
src diff: 0, pred same: 7
src diff: 2, pred same: 0
src diff: 3, pred same: 6
src diff: 1, pred same: 6
src diff: 4, pred same: 7
src diff: 4, pred same: 0
src diff: 8, pred same: 5
src diff: 2, pred same: 6
src diff: 1, pred same: 5
src diff: 2, pred same: 8
src diff: 2, pred same: 8
src diff: 0, pred same: 6
src diff: 0, pred same: 5
src diff: 2, pred same: 0
src diff: 0, pred same: 6
src diff: 0, pred same: 5
src diff: 2, pred same: 8
src diff: 0, pred same: 13
src diff: 0, pred same: 9
src diff: 1, pred same: 1
src diff: 1, pred same: 0
src diff: 2, pred same: 1
src diff: 1, pred same: 2
src diff: 2, pred same: 8
src diff: 0, pred same: 6
src diff: 0, pred same: 9
src diff: 0, pred same: 6
src diff: 1, pred same: 0
src diff: 2, pred same: 0
src diff: 1

#### Targeted Keyword Attack Analysis

In [ ]:
# targeted keyword
#target = set(["Hund"])#, "sitzt", "lehren"])   ## for Machine Translation
target = set(["police", "arrest", "wait"])   ## for Text Summarization

src_len_list = np.zeros((len(org_src_list)), dtype=int)
pred_len_list = np.zeros((len(org_pred_list)), dtype=int)
src_diff_list = np.zeros((len(org_src_list)), dtype=int)
pred_same_list = np.zeros((len(org_pred_list)), dtype=int)

keyword_success_count = 0
src_diff_total = 0
src_diff_success = 0

src_diff_percentage_total = 0.0
src_diff_percentage_success = 0.0

bleu_score_total = 0.0

for i in range(len(org_src_list)):
  org_src = org_src_list[i]
  new_src = new_src_list[i]
  org_pred = org_pred_list[i]
  new_pred = new_pred_list[i]

  # compare src, pred diff
  org_src_words = org_src.split(" ")
  new_src_words = new_src.split(" ")

  # calculate BLEU score
  score = sentence_bleu([org_src_words], new_src_words)
  bleu_score_total += score

  src_len_list[i] = len(org_src_words)
  assert(len(org_src_words)==len(new_src_words))
  src_diff_count = 0
  for j in range(src_len_list[i]):
    if org_src_words[j] != new_src_words[j]:
      src_diff_count += 1
  
  src_diff_list[i] = src_diff_count
  src_diff_total += src_diff_count

  src_diff_percentage = src_diff_count / src_len_list[i] * 100.0
  src_diff_percentage_total += src_diff_percentage

  # compare the diff (just for ref)
  org_pred_words = org_pred.split(" ")
  new_pred_words = new_pred.split(" ")
  pred_len_diff = abs(len(org_pred_words) - len(new_pred_words))
  pred_same_count = 0
  for j in range(min(len(org_pred_words), len(new_pred_words))):
    if org_pred_words[j] == new_pred_words[j]:
      if org_pred_words[j] not in overlap_count_exclude:
        pred_same_count += 1   
  
  pred_same_list[i] = pred_same_count

  #print(f"src diff: {src_diff_count}, percentage: {src_diff_count/org_src_wc*100:.2f}")
  print(f"src diff: {src_diff_count}, pred same: {pred_same_count}")
  #if pred_same_count == 1:
    #print(f"org pred: {org_pred}\nnew_pred: {new_pred}")

  # check keyword
  check_keyword = target.difference(set(new_pred_words))
  if len(check_keyword) == 0:
    keyword_success_count += 1
    src_diff_success += src_diff_count
    src_diff_percentage_success += src_diff_percentage
  else:
    print(new_pred)
  

print(f"keyword_success_count: {keyword_success_count}, success %: {keyword_success_count/len(org_src_list)*100:.2f}%")
print(f"avg src diff for all: {sum(src_diff_list)/len(src_diff_list):.2f}")
print(f"avg src diff percentage for all: {src_diff_percentage_total/len(src_diff_list):.2f}%")
print(f"avg src diff for success: {src_diff_success }")#/ keyword_success_count :.2f}")#len(src_diff_list)}")
print(f"avg src diff percentage for success: {src_diff_percentage_success}")#/keyword_success_count:.2f}%")
print(f"bleu score: {bleu_score_total/len(org_src_list):.3f}")

src diff: 3, pred same: 4
cambodian leader rejects opposition police
src diff: 3, pred same: 1
cambodian king declines to meet police summit
src diff: 3, pred same: 7
cambodian opposition asks adb to stop loans
src diff: 3, pred same: 3
cambodian ruling party warns of jail term for hun sen 's human rights record
src diff: 4, pred same: 2
cambodian opposition party says it alone must trade top position
src diff: 2, pred same: 0
cambodia 's squabbling parties break three-month deadlock
src diff: 2, pred same: 7
cambodian politicians say new partnership will not end in more violence
src diff: 4, pred same: 2
cambodian leader refuses police political detention
src diff: 2, pred same: 4
cambodian opposition leader seeks more security guarantees
src diff: 3, pred same: 5
king sihanouk praises arrest of political parties
src diff: 3, pred same: 0
honduras braces for possible catastrophe
src diff: 6, pred same: 0
western caribbean to punish honduras
src diff: 2, pred same: 0
a third day in hon

### Sentiment Analysis

In [ ]:
import requests

In [ ]:
url = "https://api.deepai.org/api/sentiment-analysis"
headers={'api-key':'-----------------------------'}  ### hide the key on github for security purpose

format = """
curl \
    -F 'text=YOUR_TEXT_HERE' \
    -H 'api-key:-----------------------------' \
    https://api.deepai.org/api/sentiment-analysis 
"""
org_src_result = []
new_src_result = []
diff = 0

for i in range(len(org_src_list)):
  org_src = org_src_list[i]
  new_src = new_src_list[i]

  # org src
  payload = dict(text=org_src)
  res = requests.post(url, data=payload, headers=headers)
  org_sent = res.text.split('"')[7]
  org_src_result.append(org_sent)
  
  # new src
  payload = dict(text=new_src)
  res = requests.post(url, data=payload, headers=headers)
  new_sent = res.text.split('"')[7]
  new_src_result.append(new_sent)

  print(f"org: {org_sent}, new: {new_sent}")
  if org_sent != new_sent:
    diff += 1
  
print(f"diff: {diff/len(org_src_list)*100:.2f}%")

org: Neutral, new: Neutral
org: Negative, new: Negative
org: Neutral, new: Neutral
org: Negative, new: Negative
org: Neutral, new: Neutral
org: Negative, new: Negative
org: Negative, new: Neutral
org: Negative, new: Negative
org: Neutral, new: Neutral
org: Negative, new: Negative
org: Positive, new: Positive
org: Negative, new: Neutral
org: Neutral, new: Neutral
org: Neutral, new: Neutral
org: Negative, new: Negative
org: Neutral, new: Neutral
org: Neutral, new: Neutral
org: Negative, new: Negative
org: Negative, new: Negative
org: Negative, new: Negative
org: Neutral, new: Neutral
org: Neutral, new: Neutral
org: Neutral, new: Neutral
org: Negative, new: Neutral
org: Neutral, new: Positive
org: Negative, new: Negative
org: Positive, new: Neutral
org: Negative, new: Neutral
org: Neutral, new: Negative
org: Positive, new: Positive
org: Positive, new: Positive
org: Negative, new: Negative
org: Neutral, new: Positive
org: Negative, new: Negative
org: Negative, new: Neutral
org: Negative, n